In [14]:
import arcpy
import os
import pandas as pd
import re

In [15]:
arcpy.env.overwriteOutput = True

In [16]:
# Set data product name
product = input("Enter the name of the data product: ")

Enter the name of the data product: SMERGE


In [17]:
# Set the input directory
input_dir = input("Enter the input directory: ")

Enter the input directory: E:\share\BIgRun\Smerge


In [18]:
# Set the output directory
output_dir = input("Enter the output directory: ")

Enter the output directory: E:\share\BIgRun\Smerge_layer


In [19]:
# Set the layer name
layer_name = input("Enter the layer name to be extracted: ")

Enter the layer name to be extracted: RZSM


In [20]:
# Read list of dates
text_file = open("dates.txt", "r")

while not os.path.exists("dates.txt"):
    print('The date list txt was not found, should be in the same directory as the .py script')
    co = input("Press enter to try again")
date = text_file.read().split(',')

offset = input('Enter the date offset, if there is none enter 0: ')
try:
    offset = int(offset)
except:
    while offset.isnumeric():
        offset = input('Offset given was not an integer value, please try again: ')

Enter the date offset, if there is none enter 0: 0


In [21]:
# Month Lag
lag = input("Enter the number of month lag: ")
lag = int(lag)
# File type
ftype = input("Enter file extention: ")
# Filename seperator
sep = input("Enter filename seperator: ")
#Date postiton
d_pos = input('Enter postion of the date with in the filenames, starting at index 0, example ALB_2000123 postion is 1:')
d_pos = int(d_pos)
# Year batch
yer = input("Enter the year you wish to process: to process all leave blank and press enter: ")
# Date Formating
d_f = input('If file use "%Y%j" type A, if "%Y%m%y" type B: ')
while d_f != 'A' and d_f != 'B':
    d_f = input('Input ERROR, If file use "%Y%j" type A, if "%Y%m%y" type B: ')
if d_f == 'A':
    d_format = '%Y%j'
if d_f == 'B':
    d_format = '%Y%m%d'

Enter the number of month lag: 0
Enter file extention: .nc4
Enter filename seperator: _
Enter postion of the date with in the filenames, starting at index 0, example ALB_2000123 postion is 1:7
Enter the year you wish to process: to process all leave blank and press enter: 
If file use "%Y%j" type A, if "%Y%m%y" type B: B


In [25]:
# Create a list of .hdf files in the input directory
hdf_files = []
for d in date:
    g = 0
    date_n = pd.to_datetime(d, format="%m/%d/%Y")
    date_n = date_n + pd.DateOffset(months=lag)
    u_date_n = date_n + pd.DateOffset(days=offset)
    l_date_n = date_n - pd.DateOffset(days=offset)
    year = date_n.year
    if d_f == 'A':
        doy = date_n.timetuple().tm_yday
        u_date_c = str(u_date_n.year) + str(u_date_n.timetuple().tm_yday).zfill(3)
        l_date_c = str(l_date_n.year) + str(l_date_n.timetuple().tm_yday).zfill(3)
    if d_f == 'B':
        u_date_c = str(u_date_n.year)  + str(u_date_n.month).zfill(2) + str(u_date_n.day).zfill(2)
        l_date_c = str(l_date_n.year)  + str(l_date_n.month).zfill(2) + str(l_date_n.day).zfill(2)
    for file in os.listdir(input_dir):
        if file.endswith(ftype):
            file_p = file.split(sep)
            file_p = file_p[d_pos]
            file_p = file_p.split('.')
            file_p = re.sub("\D", "", file_p[0])
            date_p = pd.to_datetime(file_p, format=d_format)
            if date_p.year == year and (int(l_date_c) <= int(file_p) and int(file_p) <= int(u_date_c)):
                print(file)
                hdf_files.append(os.path.join(input_dir, file))
                g = 1
                break;
        if g == 1:
            g = 0
            break;

Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160404.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160411.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160418.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160425.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160502.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160509.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160516.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160523.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160530.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160606.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160613.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160620.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160627.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160704.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160711.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160718.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160725.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160801.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160808.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160815.nc4
Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160

In [26]:
i = 0
# Iterate through the list of .hdf files and perform the subset
for hdf_file in hdf_files:
    #converts the date format in to YYYYmmdd
    date_in = pd.to_datetime(date[i], format="%m/%d/%Y")
    #Adds Monthlyy lag
    date_in = date_in + pd.DateOffset(months=lag)
    doy_in = date_in.timetuple().tm_yday
    #joins the locat and file name
    output_file = os.path.join(output_dir, product + '_'+str(date_in.year)+str(doy_in).zfill(3)+'.tif')
    if str(yer) in hdf_file:
        print(output_file)
        print(hdf_file)
        # Perform the subset
        arcpy.SubsetMultidimensionalRaster_md(hdf_file,output_file, layer_name)
    i = i + 1

    print("Process complete")

E:\share\BIgRun\Smerge_layer\SMERGE_2016095.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160404.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016102.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160411.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016109.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160418.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016116.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160425.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016123.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160502.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016130.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160509.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016137.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20160516.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2016144.tif


Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2017303.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20171030.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018092.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180402.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018099.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180409.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018106.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180416.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018113.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180423.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018120.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180430.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SMERGE_2018127.tif
E:\share\BIgRun\Smerge\Smerge_Noah_CCI_L4_RZSM0_40cm_V2.0_20180507.nc4
Process complete
E:\share\BIgRun\Smerge_layer\SM

In [15]:
i = 0
for hdf_file in hdf_files:
    #converts the date format in to YYYYmmdd
    date_in = pd.to_datetime(date[i], format="%m/%d/%Y")
    doy_in = date_in.timetuple().tm_yday
    #joins the locat and file name
    output_file = os.path.join(output_dir, product + '_'+str(date_in.year)+str(doy_in)+'.tif')
    if str(yer) in str(date[i]):
        print(date[i])
        print(output_file)
        print(hdf_file)
    i = i + 1

04/01/2019
E:\share\BIgRun\Albedo_layer\ALB_201991.tif
E:\share\BIgRun\Albedo_Merged\MCD43A3_A2019294.crf
04/08/2019
E:\share\BIgRun\Albedo_layer\ALB_201998.tif
E:\share\BIgRun\Albedo_Merged\MCD43A3_A2019301.crf


In [27]:
len(hdf_files)

99